# 離反予測を用いた SageMaker Pipelines の ML パイプライン構築

## シナリオ

電話回線の離反データセット（回線ごとのデータと離反した/しなかったの結果が残る）を使って、
SageMaker Pipelines を用いたML パイプラインを構築します。
データの詳細については[こちら](https://github.com/aws-samples/amazon-sagemaker-examples-jp/blob/master/xgboost_customer_churn/xgboost_customer_churn.ipynb)に詳細があります。  

3333 行の 元データを 1111 行ずつ 3 分割し、それぞれ 1 日目に入手するデータ、 2 日目に入手するデータ、 3 日目に入手するデータと仮定し、
* 1 日目は今あるデータを SageMaker Processing, Training, hosting をそれぞれ手動で動かす。
* 2 日目は 1 日目のデータに加えて、 2 日目に手に入ったデータも利用して学習し、 1 日目と 2 日目のモデルを比較して、2 日目のほうが精度がよければ 2 日目のモデルを hosting するのを、パイプラインを構築して実行する。
* 3 日目は 1 日目と 2 日目のデータに加えて、3 日目に手に入ったデータも利用して学習し、2 日目と 3 日目のモデルを比較して、3 日目のほうが精度がよければ 3 日目のモデルを hosting するのを、2 日目に作成したパイプラインのパラメータだけを変更して実行する。
* 4 日目は精度が下がる場合のテストとして、ノイズデータを加えて学習し、パイプラインで精度が落ちたときは モデルが更新 されないことを確認する。

In [ ]:
import os, json, sagemaker, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.inputs import TrainingInput
from sagemaker import get_execution_role
from sagemaker.workflow.parameters import ParameterInteger, ParameterString
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import ProcessingStep,TrainingStep
from sagemaker.estimator import Estimator
from sagemaker.model import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.transformer import Transformer
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep, JsonGet
from sagemaker.workflow.pipeline import Pipeline

## データ準備   
T/F の割合が変わらないように、3333 行のデータを 3 分割する。

In [ ]:
# データをダウンロード
![ -e DKD2e_data_sets.zip ] && rm DKD2e_data_sets.zip
!wget http://dataminingconsultant.com/DKD2e_data_sets.zip
!unzip -o DKD2e_data_sets.zip

In [ ]:
# 使用するデータを確認
df = pd.read_csv('./Data sets/churn.txt')
df.head()

In [ ]:
# データを分割する際、離反データが偏らないように、離反したデータと離反しなかったデータを分けて分割する
df_true = df[df['Churn?']=='True.'].reset_index()
df_false = df[df['Churn?']=='False.'].reset_index()
df_true = df_true.drop(['index'],axis=1)
df_false = df_false.drop(['index'],axis=1)

In [ ]:
# 分割前にシャッフルする
df_true_shuffle = df_true.sample(frac=1, random_state=42)
df_false_shuffle = df_false.sample(frac=1, random_state=42)

In [ ]:
# 3分割する
split_num = 3
split_df_list = []
for i in range(split_num):
    idx_min_true,idx_max_true = i*len(df_true)//split_num,(i+1)*len(df_true)//split_num
    idx_min_false,idx_max_false = i*len(df_false)//split_num,(i+1)*len(df_false)//split_num
    tmp_df = pd.concat([df_true[idx_min_true:idx_max_true],df_false[idx_min_false:idx_max_false]],axis=0)
    split_df_list.append(tmp_df)

In [ ]:
# 分割ファイルをローカルに出力する
RAWDATA_DIR = './raw_data/'
os.makedirs(f'{RAWDATA_DIR}/', exist_ok=True)
local_csvfile_list = []
for i,split_df in enumerate(split_df_list):
    file_name = f'{RAWDATA_DIR}day_{str(i+1)}.csv'
    split_df.to_csv(file_name,index=False)
    local_csvfile_list.append(file_name)
print(*local_csvfile_list)

## 一日目のデータで前処理、学習、評価、デプロイを手作業で
### 前処理
前処理は[こちら](https://github.com/aws-samples/amazon-sagemaker-examples-jp/blob/master/xgboost_customer_churn/xgboost_customer_churn.ipynb)と同じことを SageMaker Processing で行う。コンテナは scikit-learn のビルトインコンテナを利用する

In [ ]:
# Processor 定義
ROLE = get_execution_role()
PIPELINE_NAME = 'PL-test'
PRE_PROCESS_JOBNAME = f'{PIPELINE_NAME}-pre-process'
sklearn_processor = SKLearnProcessor(
    base_job_name = PRE_PROCESS_JOBNAME,
    framework_version='0.23-1',
    role=ROLE,
    instance_type='ml.m5.xlarge',instance_count=1
)

BUCKET = sagemaker.session.Session().default_bucket()
RAWDATA_SUB_PREFIX = RAWDATA_DIR.replace('./','').replace('/','')
RAWDATA_S3_URI = f's3://{BUCKET}/{PIPELINE_NAME}-{RAWDATA_SUB_PREFIX}'

# input 定義
rawcsv_s3_uri = sagemaker.s3.S3Uploader.upload(local_csvfile_list[0],RAWDATA_S3_URI)
PRE_PROCESS_RAW_DATA_INPUT_DIR = '/opt/ml/processing/input/raw_data'

# output 定義
PRE_PROCESS_TRAIN_OUTPUT_DIR = '/opt/ml/processing/output/train'
PRE_PROCESS_VALID_OUTPUT_DIR = '/opt/ml/processing/output/valid'
PRE_PROCESS_TEST_OUTPUT_DIR = '/opt/ml/processing/output/test'

sklearn_processor.run(
    code='./preprocess/preprocess.py',
    # ProcessingInput は指定したものを全て S3 から processing インスタンスにコピーされる。 Destination でコピー先を指定できる。
    inputs=[
        ProcessingInput( 
            source=rawcsv_s3_uri,
            destination=PRE_PROCESS_RAW_DATA_INPUT_DIR
        ),
    ],
    # processing インスタンスの source にあるものを全て S3 に格納する。(processing インスタンス側でこのディレクトリは自動で作成される)
    outputs=[
        ProcessingOutput(
            output_name = 'train',
            source=PRE_PROCESS_TRAIN_OUTPUT_DIR,
        ),
        ProcessingOutput(
            output_name = 'valid',
            source=PRE_PROCESS_VALID_OUTPUT_DIR,
        ),
        ProcessingOutput(
            output_name = 'test',
            source=PRE_PROCESS_TEST_OUTPUT_DIR,
        )
    ],
    # processing インスタンスのどこに csv ファイルが配置されたか、どこにファイルを出力すればよいのか、を
    # コードに渡すための引数
    arguments=[
        '--raw-data-input-dir',PRE_PROCESS_RAW_DATA_INPUT_DIR,
        '--train-output-dir',PRE_PROCESS_TRAIN_OUTPUT_DIR,
        '--valid-output-dir',PRE_PROCESS_VALID_OUTPUT_DIR,
        '--test-output-dir',PRE_PROCESS_TEST_OUTPUT_DIR,
    ]
)

### 学習
xgboost を利用する。ハイパーパラメータは[こちら](https://github.com/aws-samples/amazon-sagemaker-examples-jp/blob/master/xgboost_customer_churn/xgboost_customer_churn.ipynb)と同じにして SageMaker Training で行う。   
コンテナは xgboost のビルトインコンテナを利用する

In [ ]:
train_csv_s3_uri = sklearn_processor.latest_job.describe()['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri'] + '/train.csv'
valid_csv_s3_uri = sklearn_processor.latest_job.describe()['ProcessingOutputConfig']['Outputs'][1]['S3Output']['S3Uri'] + '/valid.csv'
test_csv_s3_uri = sklearn_processor.latest_job.describe()['ProcessingOutputConfig']['Outputs'][2]['S3Output']['S3Uri'] + '/test.csv'
print(train_csv_s3_uri)
print(valid_csv_s3_uri)
print(test_csv_s3_uri)

In [ ]:
CONTENT_TYPE='text/csv'
train_s3_input = TrainingInput(train_csv_s3_uri, content_type=CONTENT_TYPE)
valid_s3_input = TrainingInput(valid_csv_s3_uri, content_type=CONTENT_TYPE)

In [ ]:
XGB_CONTAINER_URI = sagemaker.image_uris.retrieve("xgboost", sagemaker.session.Session().boto_region_name, "1.2-1")

In [ ]:
TRAIN_JOBNAME = f'{PIPELINE_NAME}-train'
MODEL_S3_URI = f's3://{BUCKET}/{TRAIN_JOBNAME}'
HYPERPARAMETERS = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.8",
    "objective":"binary:logistic",
    "num_round":"100"
}
xgb = Estimator(
    XGB_CONTAINER_URI,
    ROLE,
    base_job_name = TRAIN_JOBNAME,
    hyperparameters=HYPERPARAMETERS,
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path = MODEL_S3_URI
)

In [ ]:
xgb.fit({'train': train_s3_input, 'validation': valid_s3_input})

### モデルの評価
今後に備えて、モデルを評価するスクリプトを作成し、動かしておく。
* 評価は AUC で行う
* SageMaker Processing を利用する
* xgboost のビルトインコンテナを利用する

In [ ]:
model_s3_uri = xgb.model_data
POST_PROCESS_JOBNAME = f'{PIPELINE_NAME}-post-process'
POST_PROCESS_INPUT_MODEL_DIR = '/opt/ml/processing/input/model'
POST_PROCESS_INPUT_DATA_DIR = '/opt/ml/processing/input/data'
POST_PROCESS_OUTPUT_DIR = '/opt/ml/processing/output'

eval_processor = ScriptProcessor(
    base_job_name = POST_PROCESS_JOBNAME,
    image_uri=XGB_CONTAINER_URI,
    command=["python3"],
    instance_type='ml.m5.xlarge',
    instance_count=1,
    role=ROLE,
)
eval_processor.run(
    code = './postprocess/postprocess.py',
    inputs=[
        ProcessingInput( 
            source=test_csv_s3_uri,
            destination=POST_PROCESS_INPUT_DATA_DIR
        ),
        ProcessingInput(
            source=model_s3_uri,
            destination=POST_PROCESS_INPUT_MODEL_DIR
        )
    ],
    outputs=[
        ProcessingOutput(
            source=POST_PROCESS_OUTPUT_DIR,
        )
    ],
    arguments=[
        '--input-model-dir',POST_PROCESS_INPUT_MODEL_DIR,
        '--input-data-dir',POST_PROCESS_INPUT_DATA_DIR,
        '--output-dir',POST_PROCESS_OUTPUT_DIR,
    ]
)


## 2 日目はパイプラインを作成する
新しくデータが入ってくるので、追加データも併せてモデルを学習しなおして精度を確認し、精度が上がっていたらモデルを交換する。
1 日目とほぼ同じことをやるので、パイプラインを作成して省力化する。
前日に追加する処理として、1 日目のデータで学習したモデルと 2 日目のデータを追加して学習したモデルで精度を比較し、精度が上がっていたらモデルを差し替える、オペレーションを追加する。

In [ ]:
# helper 関数
# pipeline で利用する名前は camel case を使うのが一般的なので、区切り文字を削除し、頭を大文字にする関数を準備
def to_camel(s_v:str,s_s:str)->str:
    '''
    s_v: camel_case に変えたい文字
    s_s: 区切り文字
    '''
    return ''.join(word.title() for word in s_v.split(s_s))

In [ ]:
rawcsv_s3_uri = sagemaker.s3.S3Uploader.upload(local_csvfile_list[1],RAWDATA_S3_URI)

In [ ]:
# 前処理ステップ定義

sklearn_processor = SKLearnProcessor(
    base_job_name = PRE_PROCESS_JOBNAME,
    framework_version='0.23-1',
    role=ROLE,
    instance_type='ml.m5.xlarge',instance_count=1
)

rawcsv_s3_uri_param = ParameterString(name='RawCsvS3Uri',default_value=rawcsv_s3_uri)

PRE_PROCESSED_TRAIN_DATA_INPUT_DIR = '/opt/ml/processing/input/train'
PRE_PROCESSED_VALID_DATA_INPUT_DIR = '/opt/ml/processing/input/valid'
PRE_PROCESSED_TEST_DATA_INPUT_DIR = '/opt/ml/processing/input/test'

# Pipeline 実行時に渡すパラメータ設定
# 名前はキャメルケース
pre_processed_train_data_s3_uri_param = ParameterString(name='PreProcessedTrainDataS3UriParam',default_value=train_csv_s3_uri)
pre_processed_valid_data_s3_uri_param = ParameterString(name='PreProcessedValidDataS3UriParam',default_value=valid_csv_s3_uri)
pre_processed_test_data_s3_uri_param = ParameterString(name='PreProcessedTestDataS3UriParam',default_value=test_csv_s3_uri)

pre_process_step = ProcessingStep(
    code='./preprocess/preprocess.py',
    name=f'{to_camel(PRE_PROCESS_JOBNAME,"-")}Step',
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(
            source=rawcsv_s3_uri_param,
            destination=PRE_PROCESS_RAW_DATA_INPUT_DIR
        ),
        ProcessingInput(
            source=pre_processed_train_data_s3_uri_param,
            destination=PRE_PROCESSED_TRAIN_DATA_INPUT_DIR
        ),
        ProcessingInput(
            source=pre_processed_valid_data_s3_uri_param,
            destination=PRE_PROCESSED_VALID_DATA_INPUT_DIR
        ),
        ProcessingInput(
            source=pre_processed_test_data_s3_uri_param,
            destination=PRE_PROCESSED_TEST_DATA_INPUT_DIR
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name = 'train',
            source=PRE_PROCESS_TRAIN_OUTPUT_DIR,
        ),
        ProcessingOutput(
            output_name = 'valid',
            source=PRE_PROCESS_VALID_OUTPUT_DIR,
        ),
        ProcessingOutput(
            output_name = 'test',
            source=PRE_PROCESS_TEST_OUTPUT_DIR,
        )
    ],
    job_arguments=[
        '--raw-data-input-dir',PRE_PROCESS_RAW_DATA_INPUT_DIR,
        '--pre-processed-train-data-input-dir',PRE_PROCESSED_TRAIN_DATA_INPUT_DIR,
        '--pre-processed-valid-data-input-dir',PRE_PROCESSED_VALID_DATA_INPUT_DIR,
        '--pre-processed-test-data-input-dir',PRE_PROCESSED_TEST_DATA_INPUT_DIR,
        '--train-output-dir',PRE_PROCESS_TRAIN_OUTPUT_DIR,
        '--valid-output-dir',PRE_PROCESS_VALID_OUTPUT_DIR,
        '--test-output-dir',PRE_PROCESS_TEST_OUTPUT_DIR,   
    ]
)

In [ ]:
# 学習ステップ定義
xgb = Estimator(
    XGB_CONTAINER_URI,
    ROLE,
    base_job_name = TRAIN_JOBNAME,
    hyperparameters=HYPERPARAMETERS,
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path = MODEL_S3_URI
)

train_step = TrainingStep(
    name=f'{to_camel(TRAIN_JOBNAME,"-")}Step',
    estimator=xgb,
    inputs={
        "train": TrainingInput(
            s3_data=pre_process_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type=CONTENT_TYPE
        ),
        "validation": TrainingInput(
            s3_data=pre_process_step.properties.ProcessingOutputConfig.Outputs[
                "valid"
            ].S3Output.S3Uri,
            content_type=CONTENT_TYPE
        )
    },
)

In [ ]:
LASTTIME_EVALUATION_FILE = 'thistime_evaluation.json'
thistime_train_eval_processor = ScriptProcessor(
    base_job_name = f'{POST_PROCESS_JOBNAME}-thistime-train-eval',
    image_uri=XGB_CONTAINER_URI,
    command=['python3'],
    instance_type='ml.m5.xlarge',
    instance_count=1,
    role=ROLE,
)
thistime_train_eval_report = PropertyFile(
    name='ThistimeTrainEvaluationReport',
    output_name='ThistimeTrainEvaluation',
    path=LASTTIME_EVALUATION_FILE
)

thistime_train_eval_step = ProcessingStep(
    code='./postprocess/postprocess.py',
    name=f'{to_camel(POST_PROCESS_JOBNAME,"-")}ThistimeTrainEvalStep',
    processor=thistime_train_eval_processor,
    inputs=[
        ProcessingInput(
            source=pre_process_step.properties.ProcessingOutputConfig.Outputs[
                'test'
            ].S3Output.S3Uri,
            destination=POST_PROCESS_INPUT_DATA_DIR
        ),
        ProcessingInput(
            source=train_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination=POST_PROCESS_INPUT_MODEL_DIR
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name='ThistimeTrainEvaluation',
            source=POST_PROCESS_OUTPUT_DIR
        ),
    ],
    property_files=[thistime_train_eval_report],
    job_arguments=[
        '--input-data-dir',POST_PROCESS_INPUT_DATA_DIR,
        '--input-model-dir',POST_PROCESS_INPUT_MODEL_DIR,
        '--output-dir',POST_PROCESS_OUTPUT_DIR,
        '--output-file',LASTTIME_EVALUATION_FILE
    ]
)

In [ ]:
LASTTIME_EVALUATION_FILE = 'lasttime_evaluation.json'
lasttime_train_eval_processor = ScriptProcessor(
    base_job_name = f'{POST_PROCESS_JOBNAME}-lasttime-train-eval',
    image_uri=XGB_CONTAINER_URI,
    command=['python3'],
    instance_type='ml.m5.xlarge',
    instance_count=1,
    role=ROLE,
)
lasttime_train_eval_report = PropertyFile(
    name='LasttimeTrainEvaluationReport',
    output_name='LasttimeTrainEvaluation',
    path=LASTTIME_EVALUATION_FILE
)
lasttime_train_model_s3_uri_param = ParameterString(
    name='lasttime-train-model-S3-URI',
    default_value=model_s3_uri
)
lasttime_train_eval_step = ProcessingStep(
    code='./postprocess/postprocess.py',
    name=f'{to_camel(POST_PROCESS_JOBNAME,"-")}LasttimeTrainEvalStep',
    processor=lasttime_train_eval_processor,
    inputs=[
        ProcessingInput(
            source=pre_process_step.properties.ProcessingOutputConfig.Outputs[
                'test'
            ].S3Output.S3Uri,
            destination=POST_PROCESS_INPUT_DATA_DIR
        ),
        ProcessingInput(
            source=lasttime_train_model_s3_uri_param,
            destination=POST_PROCESS_INPUT_MODEL_DIR
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name='LasttimeTrainEvaluation',
            source=POST_PROCESS_OUTPUT_DIR
        ),
    ],
    property_files=[lasttime_train_eval_report],
    job_arguments=[
        '--input-data-dir',POST_PROCESS_INPUT_DATA_DIR,
        '--input-model-dir',POST_PROCESS_INPUT_MODEL_DIR,
        '--output-dir',POST_PROCESS_OUTPUT_DIR,
        '--output-file',LASTTIME_EVALUATION_FILE
    ]
)

In [ ]:
model = Model(
    image_uri=XGB_CONTAINER_URI,
    model_data=train_step.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker.session.Session(),
    role=ROLE,
)
model_inputs = CreateModelInput(
    instance_type="ml.m5.large",
)
create_model_step = CreateModelStep(
    name=f'{to_camel(PIPELINE_NAME,"-")}CreateModelStep',
    model=model,
    inputs=model_inputs,
)

In [ ]:
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step=thistime_train_eval_step,
        property_file=thistime_train_eval_report,
        json_path="classification_metrics.auc.value",
    ),
    right=JsonGet(
        step=lasttime_train_eval_step,
        property_file=lasttime_train_eval_report,
        json_path="classification_metrics.auc.value",
    ),
#     right = 0.9
)

cond_step = ConditionStep(
    name=f'{to_camel(PIPELINE_NAME,"-")}ConditionStep',
    conditions=[cond_gte],
    if_steps=[create_model_step],
    else_steps=[], 
)

In [ ]:
pipeline = Pipeline(
    name=to_camel(PIPELINE_NAME,"-"),
    parameters=[
        rawcsv_s3_uri_param,
        pre_processed_train_data_s3_uri_param,
        pre_processed_valid_data_s3_uri_param,
        pre_processed_test_data_s3_uri_param,
        lasttime_train_model_s3_uri_param
    ],
    steps=[
        pre_process_step,
        train_step,
        thistime_train_eval_step,
        lasttime_train_eval_step,
        cond_step,
    ],
)

In [ ]:
definition = json.loads(pipeline.definition())
definition

In [ ]:
pipeline.upsert(role_arn=ROLE)

In [ ]:
execution = pipeline.start()

In [ ]:
execution.describe()

In [ ]:
execution.wait()

In [ ]:
# 後片付け